In [ ]:
import itertools
import logging
logging.getLogger('pgmpy').setLevel(logging.ERROR)
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, module="networkx.utils.backends")
import pandas as pd
import concurrent.futures
import random
import time
from Decom_Tree import *
import numpy as np
import networkx as nx
from scipy.linalg import sqrtm
from scipy.linalg import inv, det
from pgmpy.global_vars import logger
from pgmpy.utils import get_example_model
from pgmpy.factors.continuous import LinearGaussianCPD
from pgmpy.models import LinearGaussianBayesianNetwork

In [ ]:
def get_random_cpds(bn, loc=0, scale=0.1, inplace=False, seed=None):
    import numpy as np
    import time
    from pgmpy.factors.continuous import LinearGaussianCPD

    seed = seed if seed is not None else int(time.time() * 1000) % (2**32)
    np.random.seed(seed)

    cpds = []
    for i, var in enumerate(bn.nodes()):
        parents = bn.get_parents(var)
        cpd = LinearGaussianCPD.get_random(
            variable=var,
            evidence=parents,
            loc=loc,
            scale=scale,
            seed=seed + i
        )
        cpds.append(cpd)

    if inplace:
        bn.add_cpds(*cpds)
    else:
        return cpds


def kl_divergence(mu1, cov1, mu2, cov2):
    """
    Compute the Kullback–Leibler (KL) divergence between two multivariate Gaussian distributions.
    
    Parameters:
        mu1 (ndarray): Mean vector of the first distribution.
        cov1 (ndarray): Covariance matrix of the first distribution.
        mu2 (ndarray): Mean vector of the second distribution.
        cov2 (ndarray): Covariance matrix of the second distribution.
        
    Returns:
        float: KL divergence D_KL(N(mu1, cov1) || N(mu2, cov2))
    """
    d = len(mu1)  # Dimensionality of the distributions
    term1 = np.trace(np.dot(inv(cov2), cov1))  # Tr(Sigma_2^-1 * Sigma_1)
    term2 = np.dot(np.dot((mu2 - mu1).T, inv(cov2)), (mu2 - mu1))  # (mu2 - mu1)^T * Sigma_2^-1 * (mu2 - mu1)
    term3 = np.log(det(cov2) / det(cov1))  # log(det(Sigma_2) / det(Sigma_1))
    kl_divergence = 0.5 * (term1 + term2 - d + term3)
    return kl_divergence


def hellinger_distance(mu1, cov1, mu2, cov2):
    """
    Compute the Hellinger distance between two multivariate Gaussian distributions.
    
    Parameters:
        mu1 (ndarray): Mean vector of the first distribution.
        cov1 (ndarray): Covariance matrix of the first distribution.
        mu2 (ndarray): Mean vector of the second distribution.
        cov2 (ndarray): Covariance matrix of the second distribution.
        
    Returns:
        float: Hellinger distance H(N(mu1, cov1), N(mu2, cov2))
    """
    # Compute the determinants of the covariance matrices
    det_cov1 = det(cov1)
    det_cov2 = det(cov2)
    
    # Compute the average of the two covariance matrices
    avg_cov = (cov1 + cov2) / 2
    
    # Compute the exponent term in the Hellinger distance formula
    diff_mu = mu1 - mu2
    term = np.dot(np.dot(diff_mu.T, inv(avg_cov)), diff_mu)  # (mu1 - mu2)^T * Sigma_avg^-1 * (mu1 - mu2)
    exponent = -0.125 * term
    
    # Compute the squared Hellinger distance
    hellinger_squared = 1 - (det_cov1 ** 0.25) * (det_cov2 ** 0.25) / (det(avg_cov) ** 0.5) * np.exp(exponent)
    return np.sqrt(hellinger_squared)




def wasserstein2_distance(mu1, cov1, mu2, cov2):
    """
    Compute the 2-Wasserstein distance between two multivariate Gaussian distributions.
    
    Parameters:
        mu1 (ndarray): Mean vector of the first distribution.
        cov1 (ndarray): Covariance matrix of the first distribution.
        mu2 (ndarray): Mean vector of the second distribution.
        cov2 (ndarray): Covariance matrix of the second distribution.
        
    Returns:
        float: Wasserstein-2 distance W2(N(mu1, cov1), N(mu2, cov2))
    """
    diff_mu = mu1 - mu2
    # 均值部分
    mean_term = np.dot(diff_mu.T, diff_mu)
    
    # 协方差部分
    cov_prod_sqrt = sqrtm(np.dot(sqrtm(cov2), np.dot(cov1, sqrtm(cov2))))
    # 由于数值原因，sqrtm 可能返回复数，取实部
    cov_prod_sqrt = cov_prod_sqrt.real
    
    cov_term = np.trace(cov1 + cov2 - 2 * cov_prod_sqrt)
    
    return np.sqrt(mean_term + cov_term)


In [ ]:
sample_sizes = [500, 1000, 2500, 5000, 7500, 10000]
networks = ["ecoli70", "magic-niab", "magic-irri", "arth150"]

# 条件数汇总表格
cond_summary = pd.DataFrame(index=sample_sizes, columns=networks)

# 原有结果 DataFrame
columns = pd.MultiIndex.from_product(
    [networks, sample_sizes, ["klPQ", "hellinger", "wasserstein-2"]]
)
result_continue = pd.DataFrame(columns=columns)
repeat_num = 100

cond_all = {net: {size: [] for size in sample_sizes} for net in networks}

for file in networks:
    print(f"Processing {file}...")
    model = get_example_model(file)
    G = nx.DiGraph()
    G.add_nodes_from(model.nodes)
    G.add_edges_from(model.edges)
    decom = Graph_Decom(G)
    atoms = decom.Decom()
    
    for sample_size in sample_sizes:
        succ_num = repeat_num
        row = 0
        cond_list = []  # 存放每次实验 cov_learn 条件数
        
        while succ_num:
            # === 原 BN 拟合逻辑不动 ===
            bn = LinearGaussianBayesianNetwork()
            bn.add_nodes_from(list(G.nodes))
            bn.add_edges_from(list(G.edges)) 
            get_random_cpds(bn, inplace=True)
            df = bn.simulate(sample_size)
            
            learn_bn = LinearGaussianBayesianNetwork()
            learn_bn.add_nodes_from(list(G.nodes))
            learn_bn.add_edges_from(list(G.edges))
            learn_bn.cpds = []
            Traverse = []
            for i in atoms:
                sub_model = LinearGaussianBayesianNetwork(list(G.subgraph(list(i)).edges))
                sub_model.add_nodes_from(i)
                sub_model.cpds = []
                sub_model.fit(df[list(i)])
                for node in i:
                    if node not in Traverse:
                        if len(model.get_parents(node)) == len(sub_model.get_parents(node)):
                            Traverse.append(node)
                            learn_bn.add_cpds(sub_model.get_cpds(node))
            
            # === 新增：计算 cov_learn 条件数 ===
            _, cov_learn = learn_bn.to_joint_gaussian()
            cond_number = np.linalg.cond(cov_learn)
            cond_list.append(cond_number)
            cond_all[file][sample_size].append(cond_number)
            
            # === 原 KL/Hellinger/W2 逻辑不动 ===
            mean, cov = bn.to_joint_gaussian()
            mean_learn, cov_learn = learn_bn.to_joint_gaussian()
            kl_value = kl_divergence(mean, cov, mean_learn, cov_learn)
            hellinger_value = hellinger_distance(mean, cov, mean_learn, cov_learn)
            wasserstein_value = wasserstein2_distance(mean, cov, mean_learn, cov_learn)

            for metric_name, value in [("klPQ", kl_value), ("hellinger", hellinger_value), ("wasserstein-2", wasserstein_value)]:
                result_continue.loc[row + 1, (file, sample_size, metric_name)] = round(value, 4)
                
            succ_num -= 1
            row += 1
        
        # 汇总条件数 → mean ± std， 条件数的均值，方差
        mean_cond = np.mean(cond_list)
        std_cond = np.std(cond_list)
        cond_summary.loc[sample_size, file] = f"{mean_cond:.2e} ± {std_cond:.2e}"



In [ ]:
#转化条件数为5%，95%格式
cond_quantiles = pd.DataFrame(index=sample_sizes, columns=networks)

for net in networks:
    for size in sample_sizes:
        cond_list = cond_all[net][size]
        q05, q95 = np.percentile(cond_list, [5, 95])
        # 格式化为 (5%, 95%)
        cond_quantiles.loc[size, net] = f"({q05:.2f}, {q95:.2f})"

# 查看结果
cond_quantiles